In [3]:
import os
import pandas as pd
import requests 
from AlphanumericsTeam.data.util import get_aug_oxford_df, filter_df_regions


REPO_ROOT = os.path.abspath(os.path.join(os.path.abspath(''), os.pardir, os.pardir, os.pardir))

In [8]:
#sample input and output

# IP until 30 sep
EXAMPLE_INPUT_FILE = os.path.join(REPO_ROOT, "covid_xprize/validation/data/2020-09-30_historical_ip.csv")
prediction_input_df = pd.read_csv(EXAMPLE_INPUT_FILE,
                                  parse_dates=['Date'],
                                  dtype={"RegionName": str},
                                  encoding="ISO-8859-1")

#print(prediction_input_df)

#prediction from Aug 1 to Aug 4
EXAMPLE_OUTPUT_FILE = os.path.join(REPO_ROOT, "2020-08-01_2020-08-04_predictions_example.csv")
prediction_output_df = pd.read_csv(EXAMPLE_OUTPUT_FILE,
                                   parse_dates=['Date'],
                                   encoding="ISO-8859-1")  


print(prediction_output_df)

     CountryName RegionName       Date  PredictedDailyNewCases  IsSpecialty
0          Aruba        NaN 2020-08-01                0.820071            0
1          Aruba        NaN 2020-08-02                0.872854            0
2          Aruba        NaN 2020-08-03                0.000000            0
3          Aruba        NaN 2020-08-04                0.000000            0
4    Afghanistan        NaN 2020-08-01               80.590128            0
..           ...        ...        ...                     ...          ...
763       Zambia        NaN 2020-08-04              172.532764            0
764     Zimbabwe        NaN 2020-08-01              178.485848            0
765     Zimbabwe        NaN 2020-08-02              142.449493            0
766     Zimbabwe        NaN 2020-08-03               84.436329            0
767     Zimbabwe        NaN 2020-08-04              199.259844            0

[768 rows x 5 columns]


In [9]:
# Input data for training

DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
df = pd.read_csv(DATA_URL,
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

# Has 6 additional columns 
# 'New Cases' 
# 'GeoID' 
# 'Holidays' 
# 'pop_2020' 
# 'area_km2' 
# 'density_perkm2'
df = get_aug_oxford_df() 
df = filter_df_regions(df)

# Final list of 180 countries and 56 regions
assert df.CountryName.unique().size == 180
assert df.RegionName.unique().size == 56 + 1 

In [10]:
df.sample(3)
df["DailyChangeConfirmedCases"] = df.groupby(["CountryName", "RegionName"]).ConfirmedCases.diff().fillna(0)
california_df = df[(df.CountryName == "United States") & (df.RegionName == "California")]
california_df[["CountryName", "RegionName", "Date", "ConfirmedCases", "DailyChangeConfirmedCases"]].tail(5)

,CountryName,RegionName,Date,ConfirmedCases,DailyChangeConfirmedCases
80187,United States,California,2020-12-19,1842557.0,40362.0
80188,United States,California,2020-12-20,1884033.0,41476.0
80189,United States,California,2020-12-21,1923887.0,39854.0
80190,United States,California,2020-12-22,NaN,0.0
80191,United States,California,2020-12-23,NaN,0.0


In [1]:
# Reload the module to get the latest changes
import xprize_predictor
from importlib import reload
reload(xprize_predictor)
from xprize_predictor import XPrizePredictor

DATA_PATH = os.path.join("data", 'OxCGRT_latest_aug.csv')
predictor = XPrizePredictor(None, DATA_PATH)
 
predictor_model = predictor.train()

model_weights_file = "models/trained_model_weights.h5"
if not os.path.exists('models'):
    os.mkdir('models')
predictor_model.save_weights(model_weights_file)

Creating numpy arrays for Keras for each country...
Numpy arrays created
Trial 0
Epoch 1/100
167/167 [==============================] - 1s 5ms/step - loss: 1.0451 - val_loss: 1.0419
Epoch 2/100
167/167 [==============================] - 1s 3ms/step - loss: 0.2863 - val_loss: 0.0686
Epoch 3/100
167/167 [==============================] - 1s 3ms/step - loss: 0.0707 - val_loss: 0.0678
Epoch 4/100
167/167 [==============================] - 0s 3ms/step - loss: 0.0702 - val_loss: 0.0865
Epoch 5/100
167/167 [==============================] - 0s 3ms/step - loss: 0.0705 - val_loss: 0.0761
Epoch 6/100
167/167 [==============================] - 1s 3ms/step - loss: 0.0701 - val_loss: 0.0722
Epoch 7/100
167/167 [==============================] - 1s 3ms/step - loss: 0.0703 - val_loss: 0.0675
Epoch 8/100
167/167 [==============================] - 1s 3ms/step - loss: 0.0694 - val_loss: 0.0675
Epoch 9/100
167/167 [==============================] - 1s 3ms/step - loss: 0.0704 - val_loss: 0.0674
Epoch 10/1

In [2]:

DATA_PATH = os.path.join("data", 'OxCGRT_latest_aug.csv')
model_weights_file = "models/trained_model_weights.h5"

import xprize_predictor
from importlib import reload
reload(xprize_predictor)

from xprize_predictor import XPrizePredictor

predictor = XPrizePredictor(model_weights_file, DATA_PATH)

NPIS_INPUT_FILE = "../../validation/data/2020-09-30_historical_ip.csv"
start_date = "2020-08-01"
end_date = "2020-08-31"
 
preds_df = predictor.predict(start_date, end_date, NPIS_INPUT_FILE)
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
0,Aruba,NaN,2020-08-01,21.675105
1,Aruba,NaN,2020-08-02,24.836970
2,Aruba,NaN,2020-08-03,16.102846
3,Aruba,NaN,2020-08-04,12.457151
4,Aruba,NaN,2020-08-05,15.594049


In [3]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv
!head predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!
CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,21.675104766448243
Aruba,,2020-08-02,24.836970274395973
Aruba,,2020-08-03,16.102845790839595
Aruba,,2020-08-04,12.457151063804371
Afghanistan,,2020-08-01,236.773026715988
Afghanistan,,2020-08-02,201.81794139355142
Afghanistan,,2020-08-03,202.14350516061455
Afghanistan,,2020-08-04,230.57077403169066
Angola,,2020-08-01,75.69991670741338


In [4]:
# Check the pediction file is valid
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

In [ ]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

In [ ]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

In [5]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-30
End date: 2021-06-28


In [6]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


In [7]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-30 to 2021-06-28...
Traceback (most recent call last):
  File "predict.py", line 68, in <module>
    predict(args.start_date, args.end_date, args.ip_file, args.output_file)
  File "predict.py", line 35, in predict
    preds_df = predictor.predict(start_date, end_date, path_to_ips_file)
  File "/home/sid/git/covid-xprize/AlphanumericsTeam/predictors/lstm/xprize_predictor.py", line 115, in predict
    back_up = add_features_df(back_up)
  File "/home/sid/git/covid-xprize/AlphanumericsTeam/predictors/lstm/util.py", line 98, in add_features_df
    df[['Holidays', 'density_perkm2']] = df.apply(applyFunc, pop_df=pop_df, axis=1)
  File "/home/sid/git/covid-xprize/venv/lib/python3.7/site-packages/pandas/core/frame.py", line 7547, in apply
    return op.get_result()
  File "/home/sid/git/covid-xprize/venv/lib/python3.7/site-packages/pandas/core/apply.py", line 180, in get_result
    return self.apply_standard()
  File "/home/sid/git/covid-xprize/venv/lib/pytho

FileNotFoundError: [Errno 2] No such file or directory: 'predictions/val_6_month_future.csv'

In [ ]:

#TESTING/DEBUGGING CODE
import pandas as pd
from util import add_features_df

df = pd.read_csv("../../validation/data/2020-09-30_historical_ip.csv",
                                parse_dates=['Date'],
                                encoding="ISO-8859-1",) 
df["RegionName"] = df["RegionName"].fillna(value="") 
add_features_df(df)